In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Загрузить датасет [1 балл]

In [ ]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
# !wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
# !tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

,author,day,id,link,month,summary,tag,title,year
15259,[{'name': 'Eric Laporte'}],21,0711.3449v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,International standards for lexicon formats ar...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Lexicon management and standard formats,2007
31586,"[{'name': 'Mohammed E. Fathy'}, {'name': 'Quoc...",20,1803.07231v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,Interest point descriptors have fueled progres...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Hierarchical Metric Learning and Matching for ...,2018
12788,"[{'name': 'Andre Viebke'}, {'name': 'Suejb Mem...",25,1702.07908v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",2,Deep learning is an important component of big...,"[{'term': 'cs.DC', 'scheme': 'http://arxiv.org...",CHAOS: A Parallelization Scheme for Training C...,2017
28821,"[{'name': 'Bingke Zhu'}, {'name': 'Yingying Ch...",26,1707.08289v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",7,Image matting plays an important role in image...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Fast Deep Matting for Portrait Animation on Mo...,2017
21521,"[{'name': 'Nicole Kraemer'}, {'name': 'Masashi...",19,0902.3347v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,The runtime for Kernel Partial Least Squares (...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Lanczos Approximations for the Speedup of Kern...,2009


Немножко запрепроцессим данные

In [ ]:
old_lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'], axis=1).tolist()

sorted(old_lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

In [ ]:
from nltk.tokenize import WordPunctTokenizer

tk = WordPunctTokenizer()
lines = [] # Tokenize, replace \n with space, lower sentence and join using space
for line in old_lines:
    new_line = ' '.join([cur_line.lower() for cur_line in tk.tokenize(line)])
    lines.append(new_line)

In [ ]:
sorted(lines, key=len)[0]

'differential contrastive divergence ; this paper has been retracted .'

In [ ]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

 ### Посчитаем все возможные n граммы [2 балла]

In [ ]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens:
# - unk represents absent tokens,
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines: list[str], n: int):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)

    for line in lines:
        tokens = line.split()
        for i, last_token in enumerate(tokens):
            tokens_tuple = tuple([UNK] * max(0, (n - 1 - (i + 1) + 1)) + tokens[max(0, i - n + 1):max(0, i) ])
            counts[tokens_tuple][last_token] += 1
        i = len(tokens)
        tokens_tuple = tuple([UNK] * max(0, (n - 1 - (i + 1) + 1)) + tokens[max(0, i - n + 1):max(0, i) ])
        counts[tokens_tuple][EOS] += 1


    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    return counts


In [ ]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

In [ ]:
dummy_counts['p', '='].total()

2

### Реализовать get_possible_next_tokens и инициализацию [2 балл]

In [ ]:
probs = {}
for prefix, contexts in dummy_counts.items():
    probs  = {prefix:{elem:contexts[elem] / contexts.total() for elem in contexts.keys()}}

In [ ]:
class NGramLanguageModel:
    def __init__(self, lines, n):
        """
        Train a simple count-based language model:
        compute probabilities P(w_t | prefix) given ngram counts

        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n

        counts = count_ngrams(lines, self.n)

        # compute token proabilities given counts
        self.probs = {}
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)

        # populate self.probs with actual probabilities
        for prefix, contexts in counts.items():
            self.probs[prefix] = {elem:contexts[elem] / contexts.total() for elem in contexts.keys()}

    def get_possible_next_tokens(self, prefix) -> dict[str, float]:
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        tokens = prefix.split()
        i = len(tokens)
        tokens_tuple = tuple([UNK] * max(0, (self.n - 1 - (i + 1) + 1)) + tokens[max(0, i - self.n + 1):max(0, i) ])


        return self.probs[tokens_tuple]

    def get_next_token_prob(self, prefix, next_token) -> float:
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0.)

In [ ]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

In [ ]:
lm = NGramLanguageModel(lines, n=3)

### Реализуйте get_next_token с сэмплингом по вероятностям и температурой. [2 балла]

In [ ]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    probs_tokens = lm.get_possible_next_tokens(prefix)
    tokens = list(probs_tokens.keys())
    if temperature == 0:
        max_prob = 0
        best_token = None
        for token in tokens:
            if probs_tokens[token] > max_prob:
                best_token = token
                max_prob = probs_tokens[token]
        return best_token
    new_probs = np.array([probs_tokens[token] ** (1 / temperature ) for token in tokens ])
    new_probs = new_probs/new_probs.sum()
    return np.random.choice(tokens, p = new_probs)


In [ ]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8400 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


In [ ]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

artificial neural network ( dnn )- based dnn baseline . _EOS_


In [ ]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

bridging the gap between the points ' method ( t - sne to randomly generate environments that are not only allows for a variety of applications , e . g ., the syntactic and semantic segmentation and classification ; this paper , we propose a new framework for the case of the art performance on a set of problems in artificial intelligence , which is a crucial role in the literature . we propose a new approach to approximate the optimal solution . we also present a new approach for learning . the proposed algorithm achieves a competitive performance . _EOS_


### Также в нашей задаче может пригодиться perplexity. Добавьте её вычисление в класс `NGramLanguageModel` [1 балл]

### Реализуйте инициализацию и get_possible_next_tokens так, чтобы получилась нграмная модель с Лапласовским сглаживанием [2 балла]

In [ ]:
class LaplaceLanguageModel(NGramLanguageModel):
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta = 0.1):
        super().__init__(lines, n)

        self.probs = {}
        counts = count_ngrams(lines, self.n)

        all_tokens = set()
        for prefix, contexts in counts.items():
            for key in contexts.keys():
                all_tokens.add(key)

        for prefix, contexts in counts.items():
            self.probs[prefix] = {elem:(contexts[elem] + delta)/ (contexts.total() + delta * len(all_tokens)) for elem in contexts.keys()}


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Будем работать с Char-Level моделями, поэтому можем позволить себе все буквы английского алфавита (даже двух регистров). [1 балл]

In [ ]:
BOS, EOS = ' ', '\n'

data = pd.read_json("./arxivData.json")
lines = data.apply(lambda row: (row['title'] + ' ; ' + row['summary'])[:512], axis=1) \
            .apply(lambda line: BOS + line.replace(EOS, ' ') + EOS) \
            .tolist()

# if you missed the seminar, download data here - https://yadi.sk/d/_nGyU2IajjR9-w

In [ ]:
# get all unique characters from lines (including capital letters and symbols)
tokens = list({char for line in lines for char in line})

tokens = sorted(tokens)
n_tokens = len(tokens)
print ('n_tokens = ',n_tokens)
assert 100 < n_tokens < 150
assert BOS in tokens, EOS in tokens

n_tokens =  136


In [ ]:
# dictionary of character -> its identifier (index in tokens list)
token_to_id = {char: idx for idx, char in enumerate(tokens)}

In [ ]:
assert len(tokens) == len(token_to_id), "dictionaries must have same size"
for i in range(n_tokens):
    assert token_to_id[tokens[i]] == i, "token identifier must be it's position in tokens list"

print("Seems alright!")

Seems alright!


In [ ]:
def to_matrix(lines, max_len=None, pad=token_to_id[EOS], dtype=np.int64):
    """Casts a list of lines into torch-digestable matrix"""
    max_len = max_len or max(map(len, lines))
    lines_ix = np.full([len(lines), max_len], pad, dtype=dtype)
    for i in range(len(lines)):
        line_ix = list(map(token_to_id.get, lines[i][:max_len]))
        lines_ix[i, :len(line_ix)] = line_ix
    return lines_ix

In [ ]:
#Example: cast 4 random names to matrices, pad with zeros
dummy_lines = [
    ' abc\n',
    ' abacaba\n',
    ' abc1234567890\n',
]
print(to_matrix(dummy_lines))



[[ 1 66 67 68  0  0  0  0  0  0  0  0  0  0  0]
 [ 1 66 67 66 68 66 67 66  0  0  0  0  0  0  0]
 [ 1 66 67 68 18 19 20 21 22 23 24 25 26 17  0]]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
dummy_input_ix = torch.as_tensor(to_matrix(dummy_lines))
def compute_mask(input_ix, eos_ix=token_to_id[EOS]):
    """ compute a boolean mask that equals "1" until first EOS (including that EOS) """
    return F.pad(torch.cumsum(input_ix == eos_ix, dim=-1)[..., :-1] < 1, pad=(1, 0, 0, 0), value=True)

print('matrix:\n', dummy_input_ix.numpy())
print('mask:', compute_mask(dummy_input_ix).to(torch.int32).cpu().numpy())
print('lengths:', compute_mask(dummy_input_ix).sum(-1).cpu().numpy())

matrix:
 [[ 1 66 67 68  0  0  0  0  0  0  0  0  0  0  0]
 [ 1 66 67 66 68 66 67 66  0  0  0  0  0  0  0]
 [ 1 66 67 68 18 19 20 21 22 23 24 25 26 17  0]]
mask: [[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
lengths: [ 5  9 15]


### Реализуйте CrossEntropyLoss [1 балл]

$$L(\hat{y},y) = -\sum\limits_k^K {y^{(k)} \log{\hat{y}} ^ {(k)}}$$

In [ ]:
def compute_loss(model, input_ix):
    """
    :param model: language model that can compute next token logits given token indices
    :param input ix: int32 matrix of tokens, shape: [batch_size, length]; padded with eos_ix
    :returns: scalar loss function, mean crossentropy over non-eos tokens
    """

    input_ix = torch.as_tensor(input_ix, dtype=torch.long)

    logits = model(input_ix[:, :-1])

    probs = torch.softmax(logits, dim=1)

    label = input_ix[:, 1:]

    mask = compute_mask(label)

    extracted_loss = -torch.log(probs.gather(-1, label.unsqueeze(-1)).squeeze(-1)) * mask

    # TODO
    # Your task: implement loss function as per formula above
    # your loss should only be computed on actual tokens, excluding padding
    # predicting actual tokens and first EOS do count. Subsequent EOS-es don't
    # you may or may not want to use the compute_mask function from above.
    return extracted_loss.sum() / mask.sum()



### Реализуйте инициализацию и метод forward. Можно разобраться с pack_padded_sequence и pad_packed_sequence [3 балла]

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class RNNLanguageModel(nn.Module):
    def __init__(self, n_tokens=n_tokens, emb_size=16, hid_size=256):
        """
        Build a recurrent language model.
        You are free to choose anything you want, but the recommended architecture is
        - token embeddings
        - one or more LSTM/GRU layers with hid size
        - linear layer to predict logits

        :note: if you use nn.RNN/GRU/LSTM, make sure you specify batch_first=True
         With batch_first, your model operates with tensors of shape [batch_size, sequence_length, num_units]
         Also, please read the docs carefully: they don't just return what you want them to return :)
        """
        super().__init__() # initialize base class to track sub-layers, trainable variables, etc.


        self.emb = nn.Embedding(n_tokens, embedding_dim=emb_size, padding_idx=token_to_id[BOS])
        self.lstm = nn.LSTM(input_size=emb_size, hidden_size=hid_size, num_layers=1, batch_first=True)
        self.lin = nn.Linear(hid_size, n_tokens)

    def forward(self, input_ix):
        """
        compute language model logits given input tokens
        :param input_ix: batch of sequences with token indices, tensor: int32[batch_size, sequence_length]
        :returns: pre-softmax linear outputs of language model [batch_size, sequence_length, n_tokens]
            these outputs will be used as logits to compute P(x_t | x_0, ..., x_{t - 1})
        """

        embs = self.emb(input_ix)
        out, _ = self.lstm(embs)
        logits = self.lin(out)
        return logits

        # output tensor should be of shape [batch_size, sequence_length, n_tokens]

    def get_possible_next_tokens(self, prefix=BOS, max_len=100) -> dict[str, float]:
        """ :returns: probabilities of next token, dict {token : prob} for all tokens """
        prefix_idxs = torch.tensor([token_to_id[token] for token in prefix[max(0, len(prefix) - max_len):]]).to('cuda')
        with torch.no_grad():
            probs = self.forward(prefix_idxs)
            token_vs_prob = nn.functional.softmax(probs)[-1]
        return {prefix[i] : token_vs_prob[i] for i in range(len(probs))}


In [ ]:
model = RNNLanguageModel()

dummy_input_ix = torch.as_tensor(to_matrix(dummy_lines))
dummy_logits = model(dummy_input_ix)

assert isinstance(dummy_logits, torch.Tensor)
assert dummy_logits.shape == (len(dummy_lines), max(map(len, dummy_lines)), n_tokens), "please check output shape"
assert not np.allclose(dummy_logits.cpu().data.numpy().sum(-1), 1), "please predict linear outputs, don't use softmax (maybe you've just got unlucky)"
print('Weights:', tuple(name for name, w in model.named_parameters()))

Weights: ('emb.weight', 'lstm.weight_ih_l0', 'lstm.weight_hh_l0', 'lstm.bias_ih_l0', 'lstm.bias_hh_l0', 'lin.weight', 'lin.bias')


In [ ]:
# test for lookahead
dummy_input_ix_2 = torch.as_tensor(to_matrix([line[:3] + 'e' * (len(line) - 3) for line in dummy_lines]))
dummy_logits_2 = model(dummy_input_ix_2)

assert torch.allclose(dummy_logits[:, :3], dummy_logits_2[:, :3]), "your model's predictions depend on FUTURE tokens. " \
    " Make sure you don't allow any layers to look ahead of current token." \
    " You can also get this error if your model is not deterministic (e.g. dropout). Disable it for this test."

### Реализовать части generate [2 балла]

In [ ]:
def score_lines(model, dev_lines, batch_size):
    """ computes average loss over the entire dataset """
    dev_loss_num, dev_loss_len = 0., 0.
    with torch.no_grad():
        for i in range(0, len(dev_lines), batch_size):
            batch_ix = to_matrix(dev_lines[i: i + batch_size])
            dev_loss_num += compute_loss(model, batch_ix).item() * len(batch_ix)
            dev_loss_len += len(batch_ix)
    return dev_loss_num / dev_loss_len

def generate(model, prefix=BOS, temperature=1.0, max_len=100):
    """
    Samples output sequence from probability distribution obtained by model
    :param temperature: samples proportionally to model probabilities ^ temperature
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    with torch.no_grad():
        while True:
            #TODO
            tokens_probs = model.get_possible_next_tokens(prefix)
            tokens = list(tokens_probs.keys())
            probs = np.array([tokens_probs[token] for token in tokens])
            if temperature == 0:
                next_token = tokens[torch.argmax(probs)]
            else:
                new_probs = np.array([prob ** (1 / temperature) for prob in probs ])
                new_probs = new_probs/new_probs.sum()
                next_token =  np.random.choice(tokens, p = new_probs)
            prefix += next_token
            if next_token == EOS or len(prefix) > max_len:
                break
    return prefix

In [ ]:
from sklearn.model_selection import train_test_split
train_lines, dev_lines = train_test_split(lines, test_size=0.25, random_state=42)

In [ ]:
batch_size = 64      # <-- please tune batch size to fit your CPU/GPU configuration
score_dev_every = 250
train_history, dev_history = [], []

model = RNNLanguageModel()
opt = torch.optim.Adam(model.parameters())

# score untrained model
dev_history.append((0, score_lines(model, dev_lines, batch_size)))
# print("Sample before training:", generate(model, 'Bridging'))

### Чуть-чуть напишите тренировку [1 балл]

In [ ]:
from IPython.display import clear_output
from random import sample
from tqdm import trange
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
opt = torch.optim.Adam(model.parameters())

batches = []
for i in range(0, len(train_lines), batch_size):
    batches.append(torch.tensor(to_matrix(train_lines[i:i + batch_size])))

for i in trange(len(train_history), 5000):
    epoch_losses = []

    for batch in batches:
        batch.to(device)
        opt.zero_grad()
        loss = compute_loss(model, batch)
        epoch_losses.append(loss)
        opt.step()

    train_history.append((i, float(np.mean(epoch_losses))))

    if (i + 1) % 50 == 0:
        clear_output(True)
        plt.scatter(*zip(*train_history), alpha=0.1, label='train_loss')
        if len(dev_history):
            plt.plot(*zip(*dev_history), color='red', label='dev_loss')
        plt.legend(); plt.grid(); plt.show()
        print("Generated examples (tau=0.5):")
        for _ in range(3):
            print(generate(model, temperature=0.5))

    if (i + 1) % score_dev_every == 0:
        print("Scoring dev...")
        dev_history.append((i, score_lines(model, dev_lines, batch_size)))
        print('#%i Dev loss: %.3f' % dev_history[-1])


In [ ]:
assert np.mean(train_history[:10], axis=0)[1] > np.mean(train_history[-10:], axis=0)[1], "The model didn't converge."
print("Final dev loss:", dev_history[-1][-1])
for i in range(10):
    print(generate(model, temperature=0.5))